In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

weather_data = pd.read_csv('data/weather2.csv')
# weather_data.to_csv("output.csv")
print weather_data.loc[5]


IOError: File data/weather2.csv does not exist

In [ ]:
weather_data[weather_data.conds == "Rain"].head()

In [ ]:
import pandasql

q = """
select ENTRIESn, station, conds
from weather_data 
where conds = 'Rain'
limit 900
"""

filtered_data_rain = pandasql.sqldf(q, locals())
filtered_data_rain.describe()

In [ ]:
import pandasql

q = """
select ENTRIESn, station, conds
from weather_data 
where conds = 'Clear'
limit 900
"""

filtered_data = pandasql.sqldf(q, locals())
filtered_data.describe()
filtered_data.conds.count()

In [ ]:
# for i in range(0, len(df.index) ):
#         if i == 0:
#             df["ENTRIESn_hourly"] = 1
#         else:
#             df["ENTRIESn_hourly"].iloc[i] = df["ENTRIESn"].iloc[i] - df["ENTRIESn"].iloc[i - 1]

In [ ]:
# import sklearn.linear_model

# model = sklearn.linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)
# model.fit(features, values, sample_weight=None)

# intercept = model.intercept_
# params = model.coef_

# model.predict(X)

In [ ]:
import numpy as np
import pandas
from sklearn.linear_model import SGDRegressor

"""
In this question, you need to:
1) Implement the linear_regression() procedure using gradient descent.
   You can use the SGDRegressor class from sklearn, since this class uses gradient descent.
2) Select features (in the predictions procedure) and make predictions.

"""

def normalize_features(features):
    ''' 
    Returns the means and standard deviations of the given features, along with a normalized feature
    matrix.
    ''' 
    means = np.mean(features, axis=0)
    std_devs = np.std(features, axis=0)
    normalized_features = (features - means) / std_devs
    return means, std_devs, normalized_features

def recover_params(means, std_devs, norm_intercept, norm_params):
    ''' 
    Recovers the weights for a linear model given parameters that were fitted using
    normalized features. Takes the means and standard deviations of the original
    features, along with the intercept and parameters computed using the normalized
    features, and returns the intercept and parameters that correspond to the original
    features.
    ''' 
    intercept = norm_intercept - np.sum(means * norm_params / std_devs)
    params = norm_params / std_devs
    return intercept, params

def linear_regression(features, values):
    """
    Perform linear regression given a data set with an arbitrary number of features.
    """
    
    ###########################
    ### YOUR CODE GOES HERE ###
    ###########################
    
    model = SGDRegressor()
    model.fit(features, values)

    intercept = model.intercept_
    params = model.coef_
    
    return intercept, params

def predictions(dataframe):
    '''
    The NYC turnstile data is stored in a pandas dataframe called weather_turnstile.
    Using the information stored in the dataframe, let's predict the ridership of
    the NYC subway using linear regression with gradient descent.
    
    You can download the complete turnstile weather dataframe here:
    https://www.dropbox.com/s/meyki2wl9xfa7yk/turnstile_data_master_with_weather.csv    
    
    Your prediction should have a R^2 value of 0.40 or better.
    You need to experiment using various input features contained in the dataframe. 
    We recommend that you don't use the EXITSn_hourly feature as an input to the 
    linear model because we cannot use it as a predictor: we cannot use exits 
    counts as a way to predict entry counts. 
    
    Note: Due to the memory and CPU limitation of our Amazon EC2 instance, we will
    give you a random subset (~50%) of the data contained in 
    turnstile_data_master_with_weather.csv. You are encouraged to experiment with 
    this exercise on your own computer, locally.
    
    If you receive a "server has encountered an error" message, that means you are 
    hitting the 30-second limit that's placed on running your program. Try using a
    smaller number of features or fewer iterations.
    '''
    ################################ MODIFY THIS SECTION #####################################
    # Select features. You should modify this section to try different features!             #
    # We've selected rain, precipi, Hour, meantempi, and UNIT (as a dummy) to start you off. #
    # See this page for more info about dummy variables:                                     #
    # http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html          #
    ##########################################################################################
    
  
    #datelist = []
    
    #for item in dataframe["DATEn"]:
        #string = item[5:7]
        #number = int(string)
        #datelist.append(number)
    
    #dataframe["current_month"] = datelist
    
    dataframe["DATEn"] = pandas.to_datetime(dataframe["DATEn"])
    dataframe["day_of_week"] = dataframe.DATEn.dt.dayofweek
    
    
    features = dataframe[['rain', 'Hour', 'day_of_week', 'precipi']]
    
    
    dummy_units = pandas.get_dummies(dataframe['UNIT'], prefix='unit')
    features = features.join(dummy_units)
    
    # Values
    values = dataframe['ENTRIESn_hourly']
    
    # Get numpy arrays
    features_array = features.values
    values_array = values.values
    
    means, std_devs, normalized_features_array = normalize_features(features_array)

    # Perform gradient descent
    norm_intercept, norm_params = linear_regression(normalized_features_array, values_array)
    
    intercept, params = recover_params(means, std_devs, norm_intercept, norm_params)
    
    predictions = intercept + np.dot(features_array, params)
    # The following line would be equivalent:
    # predictions = norm_intercept + np.dot(normalized_features_array, norm_params)
    
    #print dataframe.dtypes
    #print ":::::::::::::::::::"
    #print features.dtypes
    #print ":::::::::::::::::::"
    print dataframe.describe()
    
    return predictions

## TITANIC 79.12%

In [ ]:
import numpy
import pandas
import statsmodels.api as sm

predictions = {}
df = pandas.read_csv(file_path)
for passenger_index, passenger in df.iterrows():
    passenger_id = passenger['PassengerId']

    predictions[passenger_id] = 0

    if passenger['Sex'] == 'female':
        predictions[passenger_id] = 1

    if passenger['Age'] < 18 and passenger["Pclass"] == 1:
        predictions[passenger_id] = 1

        